如果你在一个网站上看到了 jQuery，那么抓取这个网站的数据时要格外小心。jQuery 可以 动态地创建 HTML 内容，这些内容只有在 JavaScript 代码执行之后才会显示。如果你用传 统的方法抓取页面内容，就只能获得 JavaScript 代码执行之前页面上的内容

<!-- Google Analytics -->
js代码展示
<script type="text/javascript">
      var _gaq = _gaq || [];
      _gaq.push(['_setAccount', 'UA-4591498-1']);
      _gaq.push(['_setDomainName', 'oreilly.com']);
      _gaq.push(['_addIgnoredRef', 'oreilly.com']);
      _gaq.push(['_setSiteSpeedSampleRate', 50]);
      _gaq.push(['_trackPageview']);
     (function() { var ga = document.createElement('script'); ga.type =
     'text/javascript'; ga.async = true; ga.src = ('https:' ==
     document.location.protocol ? 'https://ssl' : 'http://www') +
     '.google-analytics.com/ga.js'; var s =
     document.getElementsByTagName('script')[0];
     s.parentNode.insertBefore(ga, s); })();
</script>

2.  Ajax和动态HTML

Ajax 的全称是 Asynchronous JavaScript and XML(异步 JavaScript 和 XML)，网站不需要使用单独的页面请求就可以和 Web 服务器进行交互(收 发信息)。


In [5]:
'''
# 在python中用selenium执行js
Selenium 是一个强大的网页抓取工具，最初是为网站自动化测试而开发的。
近几年，它还被 广泛用于获取精确的网站快照，因为网站可以直接运行在浏览器中。
Selenium 可以让浏览器 自动加载网站，获取需要的数据，甚至对页面截屏，或者判断网站上是否发生了某些操作。


---------------------------------------------------------
的动作。虽然这样可以看得更清楚，
但是我更喜欢让程序在后台静静地运行，所以我用一 个叫 PhantomJS 的工具代替真实的浏览器。


---------------------------------------------------------
PhantomJS 是一个无头浏览器(headless browser)。它会把网站加载到内存并执行页面上的 JavaScript，
但是它不会向用户展示网页的图形界面。把 Selenium 和 PhantomJS 结合在一 起，
就可以运行一个非常强大的网络爬虫来轻松处理 cookie、JavaScript、header 以及任何 你需要做的
'''
'''我还是在 http://pythonscraping.com/ pages/javascript/ajaxDemo.html 
创建了一个简单的页面来运行我们的爬虫。这个页面上有一 些简单的文字，是手工敲在 HTML 代码里的，
打开页面两秒钟之后，它们就会被替换成由 Ajax 生成的内容。如果我们用传统的方法抓取这个页面，
只能获取加载页面，而我们真正 需要的信息(Ajax 执行之后的页面)却抓不到。'''



'我还是在 http://pythonscraping.com/ pages/javascript/ajaxDemo.html \n创建了一个简单的页面来运行我们的爬虫。这个页面上有一 些简单的文字，是手工敲在 HTML 代码里的，\n打开页面两秒钟之后，它们就会被替换成由 Ajax 生成的内容。如果我们用传统的方法抓取这个页面，\n只能获取加载页面，而我们真正 需要的信息(Ajax 执行之后的页面)却抓不到。'

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [9]:
# driver = webdriver.PhantomJS(executable_path='</Users/danyow/chromedriver>')

chrome_options = Options()
chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(
#     executable_path='/Users/danyow/chromedriver', 
#     options=chrome_options)
driver = webdriver.Chrome(
    executable_path='/Users/danyow/chromedriver', 
)


In [10]:

driver.get('http://pythonscraping.com/pages/javascript/ajaxDemo.html')
time.sleep(3)
print(driver.find_element_by_id('content').text)
driver.close()

Here is some important text you want to retrieve!
A button to click!


在处理规模较大的网站时可能会出问题。页面的 加载时间是不确定的，具体依赖于服务器某一毫秒的负载情况，以及不断变化的网速。虽 然这个页面只需要 2 秒多的加载时间，
但是我们设置了 3 秒的等待时间以确保页面完全加 载。一种更加高效的方法是，
让 Selenium 不断地检查某个元素是否存在，以此确定页面是 否已经完全加载，如果页面加载成功就执行后面的程序。

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [15]:
'''
下面的程序检查 ID 为 loadedButton 的按钮是否存在，以此判断页面是不是已经完全加载:
'''
driver = webdriver.Chrome(
    executable_path='/Users/danyow/chromedriver', 
)

driver.get('http://pythonscraping.com/pages/javascript/ajaxDemo.html')
try:
    # 一个定位器被用来查找 ID 为 loadedButton 的按钮:
    element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'loadedButton')))
finally:
    # 定位器还可以用来创建选择器，配合 WebDriver 的 find_element 函数使用:
    print(driver.find_element_by_id('content').text)
    driver.close()


Here is some important text you want to retrieve!
A button to click!


3. 处理重定向

客户端重定向是在服务器将页面内容发送到浏览器之前，由浏览器执行 JavaScript 完成的 页面跳转，而不是服务器完成的跳转。当使用浏览器访问页面的时候，有时很难区分这两 种重定向。由于客户端重定向执行得很快，加载页面时你甚至感觉不到任何延迟，所以会 让你觉得这个重定向就是一个服务器端重定向。

In [16]:
from selenium import webdriver
import time 
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import StaleElementReferenceException

def waitForLoad(driver):
    elem = driver.find_element_by_tag_name("html")
    count = 0 
    while True:
        count += 1
        if count>20:
            print("超过10秒，即将返回")
            return 
        time.sleep(.5)
        try:
            elem == driver.find_element_by_tag_name("html")

        except StaleElementReferenceException:
            return

driver = webdriver.Chrome(
    executable_path='/Users/danyow/chromedriver', 
)
driver.get('http://pythonscraping.com/pages/javascript/redirectDemo1.html')
waitForLoad(driver)
print(driver.page_source)


超过10秒，即将返回
<html><head>
<title>The Destination Page!</title>

</head>
<body>
This is the page you are looking for!

</body></html>


In [17]:
'''等待元素的出现和消失是 Selenium 中一个常见的任务，你也可以使用前面按钮加载示例中 的 WebDriverWait 函数。
这里提供一个 15 秒钟的时限和一个 XPath 选择器，该 XPath 选择 器寻找页面内容以完成同样的任务。'''
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

driver = webdriver.Chrome(
    executable_path='/Users/danyow/chromedriver', 
)
driver.get('http://pythonscraping.com/pages/javascript/redirectDemo1.html')
try:
    bodyElement = WebDriverWait(driver, 15).until(EC.presence_of_element_located(
        (By.XPATH, '//body[contains(text(), "This is the page you are looking for!")]')))
    print(bodyElement.text)
except TimeoutException:
    print('Did not find the element')

This is the page you are looking for!
